In [ ]:
from pystac.extensions.eo import EOExtension as eo
import pystac_client
import planetary_computer
from odc.stac import configure_rio, stac_load

import dask.distributed
import dask.utils
import numpy as np
import planetary_computer as pc
import xarray as xr
from IPython.display import display
from pystac_client import Client

from odc.stac import configure_rio, stac_load

client = dask.distributed.Client()
configure_rio(cloud_defaults=True, client=client)

catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

area_of_interest = {
    "type": "Polygon",
    "coordinates": [
        [
            [-148.56536865234375, 60.80072385643073],
            [-147.44338989257812, 60.80072385643073],
            [-147.44338989257812, 61.18363894915102],
            [-148.56536865234375, 61.18363894915102],
            [-148.56536865234375, 60.80072385643073],
        ]
    ],
}

time_of_interest = "2019-06-01/2023-06-03"
search = catalog.search(
    collections=["sentinel-2-l2a"],
    intersects=area_of_interest,
    datetime=time_of_interest
)

query = catalog.search(
    collections=["sentinel-2-l2a"],
    datetime=time_of_interest,
    query={"s2:mgrs_tile": dict(eq="10SGJ")},
)
items = list(query.items())

# Check how many items were returned
# items = search.item_collection()
print(f"Returned {len(items)} Items")

In [ ]:
resolution = 20
SHRINK = 4
if client.cluster.workers[0].memory_manager.memory_limit < dask.utils.parse_bytes("4G"):
    SHRINK = 8  # running on Binder with 2Gb RAM

if SHRINK > 1:
    resolution = resolution * SHRINK

xx = stac_load(
    items,
    chunks={"x": 2048, "y": 2048},
    patch_url=pc.sign,
    resolution=resolution,
    # force dtype and nodata
    dtype="uint16",
    nodata=0,
)

print(f"Bands: {','.join(list(xx.data_vars))}")
display(xx)

In [ ]:
def to_float(xx):
    _xx = xx.astype("float32")
    nodata = _xx.attrs.pop("nodata", None)
    if nodata is None:
        return _xx
    return _xx.where(xx != nodata)


def colorize(xx, colormap):
    return xr.DataArray(colormap[xx.data], coords=xx.coords, dims=(*xx.dims, "band"))

In [ ]:
# like .astype(float32) but taking care of nodata->NaN mapping
b05 = to_float(xx.B05)
b04 = to_float(xx.B04)
ndci = (b05 - b04) / (
    b05 + b04
)
# < This is still a lazy Dask computation (no data loaded yet)

# Get a time slice `load->compute->plot`
_ = ndci.isel(time=335).compute().plot.imshow(size=7, aspect=1.2, interpolation="bicubic")

In [ ]:
ndci_comp = ndci.isel(time=0).compute()
for i in range(1,len(ndci)):
    a = ndci.isel(time=i).compute()
    ndci_comp = xr.concat([ndci_comp,a],dim="time")

In [ ]:
# Cyanobacteria Chlorophyll-a NDCI L1C https://custom-scripts.sentinel-hub.com/sentinel-2/cyanobacteria_chla_ndci_l1c/
chl = 826.57*(ndci**3) - 176.43*(ndci**2) + 19*(ndci) + 4.071
_ = chl.isel(time=1).compute().plot.imshow(size=7, aspect=1.2, interpolation="bicubic")

In [ ]:
#Floating plastic : https://www.nature.com/articles/s41598-020-62298-z#Sec10
b06 = to_float(xx.B06)
b08 = to_float(xx.B08)
b11 = to_float(xx.B11)
rp = b06 + (b11-b06)*1.87
fdi = b08 - rp
_ = fdi.isel(time=1).compute().plot.imshow(size=7, aspect=1.2, interpolation="bicubic")